In [1]:
!pip install pandas requests

In [2]:
import requests
import json
import pandas as pd

# The direct URL to the JSON file on your MinIO console
DATA_URL = 'http://localhost:9001/browser/spotify-sample/mpd.slice.0-999.json'

try:
    # 1. Fetch the data from the URL
    response = requests.get(DATA_URL)
    response.raise_for_status() # Check for HTTP errors (4xx or 5xx)

    # 2. Parse the JSON content
    data = response.json()
    
    print("Data successfully fetched and parsed!")
    
except requests.exceptions.RequestException as e:
    print(f"Error accessing the MinIO server or file: {e}")
    print("Ensure MinIO is running and accessible at http://localhost:9001.")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print("The file might not be valid JSON.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Error accessing the MinIO server or file: HTTPConnectionPool(host='localhost', port=9001): Max retries exceeded with url: /browser/spotify-sample/mpd.slice.0-999.json (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc6e163b790>: Failed to establish a new connection: [Errno 111] Connection refused'))
Ensure MinIO is running and accessible at http://localhost:9001.


In [2]:
!pip install minio


In [9]:
from minio import Minio
import json

# Khởi tạo client
client = Minio("host.docker.internal:9000", access_key="minioadmin", secret_key="minioadmin", secure=False)

# Tải file JSON từ bucket
data = client.get_object("spotify-sample", "mpd.slice.0-999.json")

# Đọc nội dung JSON
playlist_data = json.load(data)
print(len(playlist_data["playlists"]))


1000


In [11]:
playlists = playlist_data['playlists']
records = []

for p in playlists:
    for t in p['tracks']:
        records.append({
            'playlist_id': p['pid'],
            'playlist_name': p.get('name', ''),
            'track_name': t.get('track_name', ''),
            'artist_name': t.get('artist_name', ''),
            'album_name': t.get('album_name', '')
        })

df = pd.DataFrame(records)

print("✅ Số lượng playlist:", df['playlist_id'].nunique())
print("✅ Tổng số track entries:", len(df))
print("✅ Số lượng track duy nhất:", df['track_name'].nunique())
print(df.head())

✅ Số lượng playlist: 1000
✅ Tổng số track entries: 67503
✅ Số lượng track duy nhất: 30049
   playlist_id playlist_name                                  track_name  \
0            0    Throwbacks  Lose Control (feat. Ciara & Fat Man Scoop)   
1            0    Throwbacks                                       Toxic   
2            0    Throwbacks                               Crazy In Love   
3            0    Throwbacks                              Rock Your Body   
4            0    Throwbacks                                It Wasn't Me   

         artist_name                                    album_name  
0      Missy Elliott                                  The Cookbook  
1     Britney Spears                                   In The Zone  
2            Beyoncé  Dangerously In Love (Alben für die Ewigkeit)  
3  Justin Timberlake                                     Justified  
4             Shaggy                                      Hot Shot  


In [12]:
avg_tracks = df.groupby('playlist_id')['track_name'].count().mean()
print(f"Số bài hát trung bình mỗi playlist: {avg_tracks:.2f}")

# Nghệ sĩ phổ biến nhất
top_artists = df['artist_name'].value_counts().head(10)
print("\nTop 10 nghệ sĩ phổ biến nhất:")
print(top_artists)

# Album phổ biến nhất
top_albums = df['album_name'].value_counts().head(10)
print("\nTop 10 album phổ biến nhất:")
print(top_albums)

Số bài hát trung bình mỗi playlist: 67.50

Top 10 nghệ sĩ phổ biến nhất:
artist_name
Drake             939
Kanye West        415
Kendrick Lamar    385
Rihanna           350
Eminem            332
The Weeknd        296
Lil Uzi Vert      292
Ed Sheeran        285
Future            265
Chris Brown       259
Name: count, dtype: int64

Top 10 album phổ biến nhất:
album_name
Views                        234
Stoney                       184
DAMN.                        152
Greatest Hits                149
Beauty Behind The Madness    146
Original Album Classics      138
Blurryface                   134
More Life                    127
American Teen                122
The Life Of Pablo            121
Name: count, dtype: int64


In [4]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Cấu hình chuẩn để chạy với MinIO trong Docker
spark = SparkSession.builder \
    .appName("Spotify Production Pipeline") \
    .config("spark.jars", "/home/jovyan/jars/hadoop-aws-3.3.4.jar,/home/jovyan/jars/aws-java-sdk-bundle-1.12.262.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.secret.key", "minioadmin") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.caseSensitive", "true") \
    .getOrCreate()

print("Spark Session Ready!")

Spark Session Ready!


In [5]:
# 1. Định nghĩa Schema chuẩn của Spotify MPD
# Việc này giúp Spark đọc nhanh hơn và loại bỏ bản ghi sai format ngay từ đầu
track_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", LongType(), True),
    StructField("modified_at", LongType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(track_schema), True), # Nested Array
    StructField("description", StringType(), True)
])

root_schema = StructType([
    StructField("info", MapType(StringType(), StringType()), True),
    StructField("playlists", ArrayType(playlist_schema), True)
])

print("Loading Raw Data from Data Lake...")
# Đọc dữ liệu với schema đã định nghĩa
raw_df = spark.read \
    .schema(root_schema) \
    .option("mode", "PERMISSIVE") \
    .option("multiline", "true") \
    .json("s3a://spotify-raw-data/mpd.slice.0-999.json") # Demo 1 file

print(f"Raw Data Loaded.")

Loading Raw Data from Data Lake...
Raw Data Loaded.


In [6]:
print("Processing Silver Layer...")

# 1. Explode Playlists (Tách mảng playlist thành từng dòng)
playlists_df = raw_df.select(explode(col("playlists")).alias("p"))

# 2. Explode Tracks (Tách mảng tracks lồng bên trong playlist)
# Đây là bước "Explode nested track arrays" trong yêu cầu
flat_df = playlists_df.select(
    col("p.pid").alias("playlist_id"),
    col("p.name").alias("playlist_name"),
    col("p.num_tracks"),
    col("p.num_followers"),
    col("p.modified_at"),
    explode(col("p.tracks")).alias("t")
)

# 3. Parse & Normalize Metadata (Làm sạch)
# - Trim: Cắt khoảng trắng thừa
# - Lower: Chuyển về chữ thường để chuẩn hóa
# - Cast: Đảm bảo đúng kiểu dữ liệu
silver_df = flat_df.select(
    col("playlist_id"),
    trim(col("playlist_name")).alias("playlist_name"),
    col("num_tracks"),
    col("num_followers"),
    # Convert timestamp (modified_at là epoch seconds)
    to_timestamp(col("modified_at")).alias("modified_date"),
    
    # Track Metadata Parsing
    trim(col("t.track_uri")).alias("track_uri"),
    trim(col("t.track_name")).alias("track_name"),
    trim(col("t.artist_uri")).alias("artist_uri"),
    trim(col("t.artist_name")).alias("artist_name"),
    col("t.duration_ms"),
    
    # Thêm cột partition key (Ví dụ: lấy chữ cái đầu của artist để partition cho đều)
    substring(col("t.artist_name"), 0, 1).alias("artist_partition")
).drop_duplicates() # Loại bỏ dòng trùng lặp (Deduplication)

# Lưu bảng sạch này xuống Data Lake (Silver)
silver_df.write.mode("overwrite").parquet("s3a://warehouse/silver/tracks")
print("Silver Layer (Cleaned & Normalized) Saved.")
silver_df.show(3)

🔄 Processing Silver Layer...
✅ Silver Layer (Cleaned & Normalized) Saved.
+-----------+----------------+----------+-------------+-------------------+--------------------+----------------+--------------------+-------------------+-----------+----------------+
|playlist_id|   playlist_name|num_tracks|num_followers|      modified_date|           track_uri|      track_name|          artist_uri|        artist_name|duration_ms|artist_partition|
+-----------+----------------+----------+-------------+-------------------+--------------------+----------------+--------------------+-------------------+-----------+----------------+
|          0|      Throwbacks|        52|            1|2017-04-29 00:00:00|spotify:track:2eJ...|            Baby|spotify:artist:1u...|      Justin Bieber|     213973|               J|
|          1|Awesome Playlist|        39|            1|2017-09-28 00:00:00|spotify:track:2HH...|Eye of the Tiger|spotify:artist:26...|           Survivor|     243773|               S|
|     

In [7]:
print("Processing Gold Layer (Feature Engineering)...")

# Feature 1: Artist Frequency (Độ phổ biến của nghệ sĩ)
# Nghệ sĩ này xuất hiện trong bao nhiêu playlist? Tổng thời lượng nghe là bao nhiêu?
artist_features = silver_df.groupBy("artist_name") \
    .agg(
        countDistinct("playlist_id").alias("total_playlist_appearances"),
        count("track_uri").alias("total_tracks_count"),
        avg("duration_ms").alias("avg_track_duration"),
        sum("duration_ms").alias("total_listen_time_ms")
    ) \
    .withColumn("popularity_score", col("total_playlist_appearances") * 0.7 + col("total_tracks_count") * 0.3)

# Feature 2: Playlist Complexity (Độ phức tạp của Playlist)
# Playlist này có bao nhiêu nghệ sĩ khác nhau? Độ đa dạng thế nào?
playlist_features = silver_df.groupBy("playlist_id", "playlist_name") \
    .agg(
        count("track_uri").alias("playlist_length"),
        countDistinct("artist_name").alias("unique_artists"),
        avg("duration_ms").alias("avg_track_duration")
    ) \
    .withColumn("diversity_ratio", col("unique_artists") / col("playlist_length"))

print("Sample Artist Features:")
artist_features.orderBy(desc("total_playlist_appearances")).show(5)

print("Sample Playlist Features:")
playlist_features.orderBy(desc("diversity_ratio")).show(5)

Processing Gold Layer (Feature Engineering)...
Sample Artist Features:
+----------------+--------------------------+------------------+------------------+--------------------+------------------+
|     artist_name|total_playlist_appearances|total_tracks_count|avg_track_duration|total_listen_time_ms|  popularity_score|
+----------------+--------------------------+------------------+------------------+--------------------+------------------+
|           Drake|                       202|               923|237832.82665222103|           219519699|418.29999999999995|
|         Rihanna|                       170|               348| 224016.5459770115|            77957758|223.39999999999998|
|      Kanye West|                       149|               412|248709.39077669903|           102468269|227.89999999999998|
|      The Weeknd|                       139|               291| 264382.6632302406|            76935355|             184.6|
|The Chainsmokers|                       121|               2

In [8]:
print("Saving Final Data (Storage Optimization)...")

# 1. Lưu Artist Features (Partition theo chữ cái đầu để truy vấn nhanh)
# Lưu ý: Partition by artist_id/name trực tiếp sẽ tạo ra hàng triệu folder nhỏ (Bad Practice).
# Chuẩn chỉ là nên partition theo nhóm (Bucketing) hoặc chữ cái đầu.
artist_features.write \
    .mode("overwrite") \
    .partitionBy("total_playlist_appearances") \
    .parquet("s3a://warehouse/gold/artist_features")

# 2. Lưu Playlist Features (Partition by Range ví dụ theo độ dài playlist)
# Tạo bucket cột playlist_length để partition cho gọn
playlist_features_bucketed = playlist_features.withColumn(
    "length_bucket", 
    (col("playlist_length") / 50).cast("integer") * 50 # Gom nhóm 0-50, 50-100...
)

playlist_features_bucketed.write \
    .mode("overwrite") \
    .partitionBy("length_bucket") \
    .parquet("s3a://warehouse/gold/playlist_features")

print("All Data Saved & Partitioned Successfully!")

Saving Final Data (Storage Optimization)...
All Data Saved & Partitioned Successfully!
